# Implementing a Convolutional Neural Network on the CPU from Scratch
I could implement a CNN using pytorch's layers with trivial ease `nn.Conv2d` is all that is required. However, I'd like to demonstrate a detailed understanding of the fundamental mechanics.

In [1]:
import torch 
from torch import optim, nn
import torch.nn.functional as F

from PIL import Image
import numpy as np

import pandas as pd
import os

In [2]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [3]:
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
test_df.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


It's clear each row represents an image. I want to convert the underlying data structure, `numpy.ndarray` to a PNG file so that I may augment and manipulate the images with imaging libraries, like `PIL`.

In [24]:
for i in range(len(train_df)):
    arr = np.array(train_df.iloc[i,1:]).reshape(28,28)
    arr = arr.astype('uint8')

    img = Image.fromarray(arr, mode='L')

    label = train_df.iloc[i,0] 
    
    os.makedirs(f'./train/{label}/', exist_ok=True)
    img.save(f'./train/{label}/train_{i}.png')

In [8]:
for i in range(len(test_df)):
    arr = np.array(test_df.iloc[i,:]).reshape(28,28)
    arr = arr.astype('uint8')
    
    img = Image.fromarray(arr, mode='L')
    
    os.makedirs('./test/', exist_ok=True)
    img.save(f'./test/test_{i}.png')

### Let's establish baseline accuracy
A simple linear model with zero data augmentation.

In [5]:
full_ds = []

for i in range(len(train_df)):
    arr = np.array(train_df.iloc[i,1:])
    arr = arr.astype(np.float32) / 255
    features_t = torch.from_numpy(arr)
    
    label_t = torch.tensor(train_df.iloc[i, 0])
    label_t = F.one_hot(label_t, num_classes=10)
    label_t = label_t.float()
    
    full_ds.append((features_t, label_t))

In [6]:
cutoff = int(len(full_ds) * 0.8)
train_ds = full_ds[:cutoff]
val_ds = full_ds[cutoff:]

In [7]:
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=False)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=64, shuffle=False)


In [8]:
input_neurons = len(test_df.columns)
classes = len(train_df["label"].unique())

In [9]:
dense_model = nn.Sequential(nn.Linear(input_neurons, input_neurons//2),
                           nn.ReLU(),
                           nn.Linear(input_neurons//2, input_neurons//4),
                           nn.ReLU(),
                           nn.Linear(input_neurons//4, input_neurons//8),
                           nn.ReLU(),
                           nn.Linear(input_neurons//8, classes))    # I want to softmax these outputs

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(dense_model.parameters(), lr=1e-3)

In [11]:
from learner import ClassificationLearner
clearn = ClassificationLearner(model=dense_model,
                              optimizer=optimizer,
                              criterion=criterion,
                              train_dl=train_dl,
                              val_dl=val_dl)

In [12]:
from rich.console import Console
console = Console()
try:
    clearn.fit(50, save_best=True)
except Exception as e:
    console.print_exception(show_locals=True)

Epoch 0 Val Loss: 0.0105  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 1 Val Loss: 0.0088  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 2 Val Loss: 0.0087  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 3 Val Loss: 0.0048  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 4 Val Loss: 0.0023  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 5 Val Loss: 0.0016  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 6 Val Loss: 0.0008  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 7 Val Loss: 0.0005  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 8 Val Loss: 0.0001  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 9 Val Loss: 0.0022  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 10 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 11 Val Loss: 0.0007 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 12 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 13 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 14 Val Loss: 0.0358 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 15 Val Loss: 0.0001 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 16 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 17 Val Loss: 0.0005 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 18 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 19 Val Loss: 0.0003 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 20 Val Loss: 0.0001 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 21 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 22 Val Loss: 0.0001 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 23 Val Loss: 0.0010 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 24 Val Loss: 0.0001 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 25 Val Loss: 0.0004 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 26 Val Loss: 0.0001 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 27 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 28 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 29 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 30 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 31 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 32 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 33 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 34 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 35 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 36 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 37 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 38 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 39 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 40 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 41 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 42 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 43 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 44 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 45 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 46 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 47 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 48 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
Epoch 49 Val Loss: 0.0000 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /var/folders/5c/jpqgr68j18vdx0qd4s8vzlp40000gn/T/ipykernel_4263/205538372.py:4 in <cell   │
│ line: 3>                                                                                  │
│                                                                                           │
│ [Errno 2] No such file or directory:                                                      │
│ '/var/folders/5c/jpqgr68j18vdx0qd4s8vzlp40000gn/T/ipykernel_4263/205538372.py'            │
│ /Users/sfdev/Code/kaggle/mnist/learner.py:137 in fit                                      │
│                                                                                           │
│   134 │   │   │   │   logging.error(f"OSError: {ose}, saving model to ./{self.best_accura │
│   135 │   │   │   │   save_f = f"{self.best_accuracy:.4%}"                                │
│   136 │   │   │                                                                           │
│ ❱ 137 │   │   │   torch.save(self.best_model, save_f)                                     │
│   138 │   │   │   logging.info(f"Saved model to {save_f}")                                │
│   139                                                                                     │
│   140                                                                                     │
│                                                                                           │
│ ╭─────────────────────────────────────── locals ────────────────────────────────────────╮ │
│ │       acc_thresh = 0.5                                                                │ │
│ │         accuracy = 0.9903452380952381                                                 │ │
│ │            epoch = 49                                                                 │ │
│ │           epochs = 50                                                                 │ │
│ │         features = tensor([[0., 0., 0.,  ..., 0., 0., 0.],                            │ │
│ │                    │   │   [0., 0., 0.,  ..., 0., 0., 0.],                            │ │
│ │                    │   │   [0., 0., 0.,  ..., 0., 0., 0.],                            │ │
│ │                    │   │   ...,                                                       │ │
│ │                    │   │   [0., 0., 0.,  ..., 0., 0., 0.],                            │ │
│ │                    │   │   [0., 0., 0.,  ..., 0., 0., 0.],                            │ │
│ │                    │   │   [0., 0., 0.,  ..., 0., 0., 0.]])                           │ │
│ │   progress_tasks = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, ... +40]                            │ │
│ │    rich_progress = <rich.progress.Progress object at 0x12749a3e0>                     │ │
│ │        save_best = True                                                               │ │
│ │           save_f = './saved_models/99.0345%'                                          │ │
│ │             self = <learner.ClassificationLearner object at 0x12749a860>              │ │
│ │          targets = tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],                  │ │
│ │                    │   │   [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],                  │ │
│ │                    │   │   [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],                  │ │
│ │                    │   │   [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],                  │ │
│ │                    │   │   [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],                  │ │
│ │                    │   │   [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],                  │ │
│ │                    │   │   [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],                  │ │
│ │                    │   │   [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],                  │ │
│ │                    │   │   [0., 0., 0., 0., 0., 

### Progress bar didn't work like expected BUT, our model did train pretty well :)

In [13]:
clearn.best_accuracy

0.9903452380952381